In [34]:
import chardet
import pandas as pd
import csv
from enum import Enum

In [35]:
tutors_column_names = {
#     name of the tutor
    "Name" : "Full Name",
#    email of the tutor  
    "Email" : "UChicago Email Address",
#     major of the tutor
    "Major" : "Intended Major (it's okay if you're Undecided!)",
#     the subjects the tutor is comfortable helping with
    "Subjects" : "What subjects are you comfortable helping your student with?",
#     the grades of the tutor is comfortable tutoring (Comma separated i.e. 5,6,7,8)
    "Grades" : "What grade levels are you comfortable tutoring? (please check all that apply)",
#     if tutor is an international student
    "International Student" : "Are you an international student? (We need this information to ensure you're paid directly by MTM, which keeps you from violating the terms of your visa.)",
#     if tutor has a disability
    "Disability" : "Are you comfortable working with students with disabilities including but not limited to autism, ADHD, and dyslexia? (Your honesty with this question helps us ensure that all students get a tutor who can fully address their needs.)",
#     the method in which the tutor would be willing to tutor the child
    "Tutor Method" : "Would you prefer to do in-person or online (i.e. via Skype, webwhiteboard, and other programs) tutoring?",
#     if the tutor would be willing to do standardized test prep
    "Test Prep" : "Would you be willing to do standardized test prep for elementary schoolers within the one-on-one tutoring context? (We will provide the foundational resources for you.)",
#     the max number of students tutor is willing to take
    "Max Students" : "How many students would you like to take on? (Keep in mind that most families want to meet twice a week for 1-1.5 hours.)"
}

class Tutor_Method(Enum):
    IN_PERSON = 1
    ONLINE_ONLY = 2
    NO_PREFERENCE = 3
    

In [36]:
class Tutor:

    def __init__(self, tutor_dict):
        '''
        Initialize every instance of Tutor with tutor_dict, a dictionary with one tutor's informtion.
        '''
#         Strings
        self.name = tutor_dict[tutors_column_names["Name"]]
        self.subjects = tutor_dict[tutors_column_names["Subjects"]]
        self.major = tutor_dict[tutors_column_names["Major"]]
        
#         Array
        self.grades = tutor_dict[tutors_column_names["Grades"]].split(',')
    
#         Booleans
        self.intl_student = "yes" in tutor_dict[tutors_column_names["International Student"]].lower()
        self.disabl = "yes" in tutor_dict[tutors_column_names["Disability"]].lower()
        self.test_prep = "yes" in tutor_dict[tutors_column_names["Test Prep"]].lower()
        
#         Enums
        tutor_method_preference = tutor_dict[tutors_column_names["Tutor Method"]]
        if tutor_method_preference == "In-person":
            self.tutor_method = Tutor_Method.IN_PERSON
        elif tutor_method_preference == "Either is fine":
            self.tutor_method = Tutor_Method.NO_PREFERENCE
        else:
            self.tutor_method = Tutor_Method.ONLINE_ONLY
            
#           Integers
        self.max_students = int(tutor_dict[tutors_column_names["Max Students"]])
        self.num_students = 0
        self.matched_students = []

    def __repr__(self):
        return f'Tutor({self.name}), Matched students: {self.matched_students}'

    def __str__(self):
        return f'Tutor({self.name}), Matched students: {self.matched_students}'



In [49]:
class Tutor_Manager:
    '''
    Tutor manager holds and initializes all instances of tutors from a given csv.
    '''
    def __init__(self, filename):
        self.tutors = []

        #open Tutors csv and turn each row into a Tutor instance
        with open(filename, 'r') as f1:
            reader = csv.DictReader(f1)
            for row in reader:
#                 if no email, skip
                if (row[tutors_column_names["Email"]] != ""):
                    tut = Tutor(row)
                    self.tutors.append(tut)

    def __repr__(self):
        return f'{self.tutors}'

    def __str__(self):
        return f'{self.tutors}'


In [51]:
main()

[Tutor(Albert Chen), Matched students: [], Tutor(Alexander Coker), Matched students: [], Tutor(Andrea Omehe ), Matched students: [], Tutor(Anjelica Fabro), Matched students: [], Tutor(Anna Aguiar Kosicki), Matched students: [], Tutor(Antonia Stefanescu), Matched students: [], Tutor(Apoorva Krishnan), Matched students: [], Tutor(Audrey Teo), Matched students: [], Tutor(Ava Geenen), Matched students: [], Tutor(Beatrix Brandfield-Harvey), Matched students: [], Tutor(Calvin Wilder), Matched students: [], Tutor(Cavell Means), Matched students: [], Tutor(Clare Suter), Matched students: [], Tutor(Cynthia Pang), Matched students: [], Tutor(Eric Wang), Matched students: [], Tutor(Erin Stafford), Matched students: [], Tutor(Gary Zhao), Matched students: [], Tutor(Genevieve Faber), Matched students: [], Tutor(Greer Baxter), Matched students: [], Tutor(Gwendolyn Gilbert-Snyder), Matched students: [], Tutor(Han Bin Kim), Matched students: [], Tutor(Hannah Buonomo), Matched students: [], Tutor(Harry

In [52]:
guardians_column_names = {
    "First Name" : "Guardian First Name",
    "Last Name" : "Guardian Last Name",
    "Title" : "Guardian Preferred Title",
    "Email" : "Guardian Email Address",
    "Phone Number" : "Primary Phone Number",
    "Scholarship" : "Are you planning on applying for a scholarship?"
}

In [53]:
class Guardian:

    def __init__(self, guardian_dict):
        '''
        Initialize Guardian object with guardian_dict, a dictionary with guardian info.
        '''
        self.first_name = guardian_dict[guardians_column_names["First Name"]]
        self.last_name = guardian_dict[guardians_column_names["Last Name"]]
        self.title = guardian_dict[guardians_column_names["Title"]]
        self.email = guardian_dict[guardians_column_names["Email"]]
        self.phone = guardian_dict[guardians_column_names["Phone Number"]]
        self.scholarship = guardian_dict[guardians_column_names["Scholarship"]]

    def __eq__(self, other):
        return (self.first_name == other.first_name) and (self.last_name == other.last_name)

    def __repr__(self):
        return f'Guardian({self.first_name} {self.last_name}, {self.email})'

    def __str__(self):
        return f'Guardian({self.first_name} {self.last_name}, {self.email})'


In [54]:
class Student:
    '''
    Every instance of Student represents one student. It requires that Tutor_Manager be run first.
    '''
    def __init__(self, student_dict, guardian, all_tutors):
        '''
        Iniitalize a student object.

        student_dict: dictionary with student information
        Guardian: Guardian object associated with student
        all_tutors: Tutor_Manager object with list of all tutors
        '''
        
        for key in student_dict.keys():
            if 'FIRST NAME' in key.upper():
                self.first_name = student_dict[key]
            elif 'LAST NAME' in key.upper():
                self.last_name = student_dict[key]
            elif 'GRADE' in key.upper():
                self.grade = student_dict[key] #int
            elif 'SCHOOL' in key.upper():
                self.school = student_dict[key]
            elif 'SUBJECTS' in key.upper():
                self.subjects = student_dict[key]
            elif 'IN-PERSON OR ON-LINE' in key.upper():
                self.method = student_dict[key]
            elif 'TIMES A WEEK' in key.upper():
                self.frequency = student_dict[key] 
            elif 'AVAILABLE FOR TUTORING' in key.upper():
                self.availability = student_dict[key]
            elif 'PREVIOUS TUTOR' in key.upper():
                self.previous_tutor_name = student_dict[key] #if no preference, value should be 0
                self.check_previous_tutor = False #whether or not a human needs to check the student's previous tutor preference
                self.previous_tutor_match = [] #list of matches to current tutor list

            #elif 'MAROON TUTOR MATCH BEFORE' in key.upper():
                #self.return_student = student_dict[key]

        self.guardian = guardian
        self.tutor_matches = []

        if isinstance(self.previous_tutor_name, str): #student has a previous tutor preference, try to find a match within tutors list

            def find_previous_tutors(tutor_list):
                '''
                This method searches a list of all tutor objects to see if student's preferred previous tutor is within the list of current Tutor objects.
                If there are multiple matches, change self.check_previous_tutor to True.

                tutor_list: list of all Tutor objects

                returns: list of match(es) of Tutors (empty if no matches, 1 element if 1 match, multiple elements if more than 1 match)
                '''
                if not isinstance(self.previous_tutor_name, str): #self.previous_preference is 0
                    raise TypeError

                match = []

                if len(self.previous_tutor_name.split()) == 1: #previous tutor name has one name, presumably a first 
                    match = [tutor 
                             for tutor in tutor_list.tutors 
                             if tutor.name.split()[0].upper() == self.previous_tutor_name.upper()
                            ] #split tutor name by space and compare first name to self.previous_tutor_name
                
                else: #previous tutor name has more than one name, presumably a first and last
                    match = [tutor 
                             for tutor in tutor_list.tutors 
                             if tutor.name.upper() == self.previous_tutor_name.upper()
                            ]

                if len(match) > 1:
                    self.check_previous_tutor = True #there is more than one match so flag attribute as True to indicate that a human should check

                return match

            self.previous_tutor_match = find_previous_tutors(all_tutors) #all_tutors is the list of all tutors from Tutor_Manager


    def __repr__(self):
        return f'Student({self.first_name}, {self.last_name})'

    def __str__(self):
        return f'Student({self.first_name}, {self.last_name})'


In [55]:
class Student_Manager:
    '''
    Student_Manager holds and initializes all instances of students from a given csv.
    '''
    def __init__(self, filename, all_tutors):
        '''
        Parse the csv file and initialize Student objects within the file.

        filename: csv with student info
        all_tutors: Tutor_Manager object with list of all tutors
        '''
        self.students = []

        with open(filename, 'rb') as f:
            result = chardet.detect(f.read()) #identify encoding code needed for pandas read_csv
            
        student_dict = pd.read_csv(filename, encoding=result['encoding']) #create dataframe of csv 
        student_dict.fillna(0, inplace = True) 
        all_col_names = list(student_dict) #list of dataframe column names

        def add_students(df):
            '''
            This function takes the pandas dataframe generated from the csv file and parses it to
            find all students in the file.

            returns: list of Student objects
            '''
            students = []
            for i, row in df.iterrows(): 
                #row is a panda Series

                def guardian_info():
                    '''
                    Identify columns and values relating to the guardian of the student(s).

                    returns: dictionary with guardian-relevant keys and values
                    '''
                    guardian_dict = {} 
                    for col_name in all_col_names: 
                        if 'scholarship' in col_name.lower() or 'guardian' in col_name.lower() or 'phone' in col_name.lower(): #check for colnames with Guardian info 
                            guardian_dict[col_name] = row.get(col_name)
                    return guardian_dict

                #create Guardian of all student(s) in this row
                guardian_dict = guardian_info()
                student_guardian = Guardian(guardian_dict) #create Guardian of all students in this row

                #start parsing students
                for i in range(1, 5): #each row has max 4 students
                    name = f'Student {i} First Name'
                    student_i = df.columns.get_loc(name) #index of Student's first name
                    another_student_col = 'Do you have another student applying to MTM?' #column between students
                    
                    if i==1: #Student one
                        end_student_i = df.columns.get_loc(another_student_col) #Student one ends at first column that asks Do you have another student..?
                    
                    elif i==2: #Student two
                        if str(row.get(another_student_col)).upper() == 'YES' or row.get(name): #second student exists
                            next_student_col = another_student_col + f'.{i-1}'
                            end_student_i = df.columns.get_loc(next_student_col) #end of Student 2 ends with 'Do you have another student..?.1'
                        else:
                            break
                    
                    else: #Student three or four
                        if str(row.get(another_student_col + f'.{i-2}')).upper() == 'YES' or row.get(name):
                            next_student_col = another_student_col + f'.{i-1}'
                            end_student_i = df.columns.get_loc(next_student_col)
                        else:
                            break

                    student_dict = {}

                    #for each column in the range of the given student columns, add key and value to empty student_dict
                    for i in range(student_i, end_student_i):
                        col_name = df.columns[i] #find column name associated with column index
                        val = row.get(col_name) #get value in the row for that column name
                        student_dict[col_name] = val #add column name as key and set value

                    students.append(Student(student_dict, student_guardian, all_tutors))

            return students

        self.students = add_students(student_dict)

    def match_guardian(self, Guardian):
        '''
        This method takes a guardian name and finds all of the students associated with that guardian.

        guardian: Guardian object

        returns: list of student(s) with that guardian
        '''
        return [i 
                for i in self.students 
                if i.guardian == Guardian
               ]

    def __repr__(self):
        return f'{self.students}'

    def __str__(self):
        return f'{self.students}'

In [ ]:
def main(filename1 = 'tutor.csv', filename2 = 'student.csv'):
all_tutors = Tutor_Manager(filename1)
	all_students = Student_Manager(filename2)
